In [2]:
import pandas as pd
import numpy as np

data = pd.read_excel(r"E:\HKUST_CLASS\22spring\Investment_Model\project1_simplefactor\DATA.xlsx")


def weighted_avg(x, weights):
    weights.iloc[x.isnull()==True]=0
    x = x.fillna(0)
    return sum(x*weights)/sum(weights)



In [16]:
#Calculate Sector Return 下个月的return
Sec_perf = data.groupby(['DT','GICS'])[['SUBS_1M_USD','WEIGHT']].apply(lambda x: weighted_avg(x['SUBS_1M_USD'],weights=x['WEIGHT'])).to_frame()
Sec_perf = Sec_perf.unstack(level = 1)
Sec_perf.columns = Sec_perf.columns.droplevel()
Sec_perf.columns = ['Energy','Materials','Industrials','Consumer Discretionary','Consumer Staples','Health Care','Financials',\
'Information Technology','Telecommunication Services','Utilities','Real Estate']
Sec_perf.head()

,Energy,Materials,Industrials,Consumer Discretionary,Consumer Staples,Health Care,Financials,Information Technology,Telecommunication Services,Utilities,Real Estate
DT,,,,,,,,,,,
2015-05-29,-0.030290,-0.025832,-0.024961,0.005511,-0.024884,-0.003726,0.001871,-0.052625,0.002775,-0.063355,-0.042408
2015-06-30,-0.072497,-0.047884,0.001456,0.065216,0.048341,0.024479,0.024226,0.009081,0.100859,0.059242,0.058059
2015-07-31,-0.054844,-0.062339,-0.059107,-0.041046,-0.062312,-0.078523,-0.071212,-0.064608,-0.045771,-0.040818,-0.049555
2015-08-31,-0.057201,-0.076217,-0.019176,-0.004049,0.004182,-0.052035,-0.040113,-0.010265,-0.015458,0.023441,0.019790
2015-09-30,0.115982,0.134579,0.093988,0.089543,0.043078,0.067770,0.058202,0.094477,0.120623,0.015297,0.065474


In [17]:
#Calculate Short Interest Return 下个月的return
nobas=3
data['basket']=data.groupby('DT')['3M_AVG_RATIO'].rank(method='average',pct=True,ascending=True)
data['basket']=data['basket'].apply(lambda x : np.trunc((x-0.00000001)*nobas)+1)
perf=data.groupby(['DT','basket'])[['SUBS_1M_USD','WEIGHT']].apply(lambda x:weighted_avg(x['SUBS_1M_USD'], weights=x['WEIGHT'])).to_frame()
perf.columns=['perf_port']
perf.head()

perf_port
DT         basket           
2015-05-29 1.0     -0.015153
           2.0     -0.025802
           3.0     -0.015253
2015-06-30 1.0      0.037415
           2.0      0.016943

In [19]:
# 找到每个月long的股票,算weight
STOCK= data.copy(deep=True)
STOCK["DT"]=pd.to_datetime(STOCK["DT"]).dt.date
# 思路：先按DT分类，basket1计算weight，其他weight为0
perf_sum=STOCK.groupby(['DT','basket'])[['WEIGHT']].transform('sum')
STOCK.WEIGHT=STOCK.WEIGHT.fillna(0)
STOCK["perf_sum"]=perf_sum
STOCK["perf_w"]=''
for i in range(STOCK.shape[0]):
    if STOCK.loc[i,'basket']==1:
        STOCK.loc[i,"perf_w"]= STOCK.loc[i,"WEIGHT"]/STOCK.loc[i,"perf_sum"]
    else:
        STOCK.loc[i,"perf_w"]=0
STOCK.head()

,Unnamed: 0,DT,STOCK_NAME,WEIGHT,SUBS_1M_USD,3M_AVG_RATIO,GICS,basket,perf_sum,perf_w
0,0,2015-05-29,LYB UN Equity,0.002600,0.023937,2.502230,15,2.0,0.320630,0
1,1,2015-05-29,SBNY UW Equity,0.000382,0.048264,2.692720,40,2.0,0.320630,0
2,2,2015-05-29,AXP UN Equity,0.004401,-0.025088,1.784833,40,1.0,0.477757,0.009211
3,3,2015-05-29,VZ UN Equity,0.010958,-0.057241,4.843214,50,3.0,0.195251,0
4,4,2015-05-29,AVGO UW Equity,0.002065,-0.102249,0.747424,45,1.0,0.477757,0.004321


In [31]:
W=STOCK[["DT","STOCK_NAME","perf_w"]]
W=W.tail(18870)
outputpath=r"E:\HKUST_CLASS\22spring\Investment_Model\project2\WEIGHT.xlsx"
W.to_excel(outputpath,sheet_name="Sheet1",index="True",header="True")

In [22]:
perf_bm=data.groupby(['DT'])[['SUBS_1M_USD','WEIGHT']].apply(lambda x:weighted_avg(x['SUBS_1M_USD'], weights=x['WEIGHT'])).to_frame()
perf_bm.columns=['perf_bm']
perf=perf.merge(perf_bm,how='right',left_index=True,right_index=True)
perf['perf_rel']=perf.perf_port-perf.perf_bm
 
perf=perf['perf_rel'].unstack(level = 1)
perf.columns= ['Low','Mid','High']
perf['LS']=perf.Low-perf.High

In [23]:
# Long only porfolio's subsequent monthly return & Benchmark Return
P_Rtn= perf["Low"].copy(deep=True)
P_Rtn=pd.DataFrame(P_Rtn)
P_Rtn.head()


,Low
DT,
2015-05-29,0.003456
2015-06-30,0.010207
2015-07-31,0.003870
2015-08-31,-0.002068
2015-09-30,0.019559


In [24]:
perf = perf.drop(columns=['Low','Mid','High'])
perf = perf.reset_index()
perf.columns= ['DT','Short Interest']
perf = perf.merge(P_Rtn,how='left',left_on='DT',right_on='DT')
perf.head()

,DT,Short Interest,Low
0,2015-05-29,0.000100,0.003456
1,2015-06-30,0.017323,0.010207
2,2015-07-31,0.004415,0.003870
3,2015-08-31,-0.001104,-0.002068
4,2015-09-30,0.042487,0.019559


In [25]:
#Calculate Market and Macro blocks
Factor = pd.read_excel(r"E:\HKUST_CLASS\22spring\Investment_Model\project2\BBG_Monthly_Factor.xlsx")

for i in range(Factor.shape[0]-1):
    #MoM change for treasury yield
    Factor.loc[i,'USGG10YR Index'] = (Factor.loc[i+1,'USGG10YR Index']-Factor.loc[i,'USGG10YR Index'])/100
    Factor.loc[i,'USGG12M Index'] = (Factor.loc[i+1,'USGG12M Index']-Factor.loc[i,'USGG12M Index'])/100
    
    #Percentage change for the rest factor
    Factor.loc[i,'SPX Index'] = (Factor.loc[i+1,'SPX Index']-Factor.loc[i,'SPX Index'] )/Factor.loc[i,'SPX Index']
    Factor.loc[i,'VIX Index'] = (Factor.loc[i+1,'VIX Index']-Factor.loc[i,'VIX Index'] )/Factor.loc[i,'VIX Index']
    Factor.loc[i,'SPX Index Volume'] = (Factor.loc[i+1,'SPX Index Volume']-Factor.loc[i,'SPX Index Volume'] )/Factor.loc[i,'SPX Index Volume']
    Factor.loc[i,'USO US Equity'] = (Factor.loc[i+1,'USO US Equity']-Factor.loc[i,'USO US Equity'] )/Factor.loc[i,'USO US Equity']
    Factor.loc[i,'CONSSENT Index'] = (Factor.loc[i+1,'CONSSENT Index']-Factor.loc[i,'CONSSENT Index'] )/Factor.loc[i,'CONSSENT Index']

Factor = Factor.drop(Factor.index[-1])
Factor.head()


,DT,SPX Index,VIX Index,SPX Index Volume,USGG10YR Index,USGG12M Index,USO US Equity,CONSSENT Index
0,2015-05-29,-0.021012,0.317197,0.201474,0.002316,0.000204,-0.021172,0.059537
1,2015-06-30,0.019742,-0.335162,-0.010625,-0.001730,0.000459,-0.215795,-0.031217
2,2015-07-31,-0.062581,1.345710,0.188683,0.000379,0.000664,0.019243,-0.012889
3,2015-08-31,-0.026443,-0.138234,0.060241,-0.001812,-0.000664,-0.076149,-0.051143
4,2015-09-30,0.082983,-0.384898,-0.059853,0.001054,0.000102,0.008856,0.032110


In [26]:
#Combine Market, Macro, Style and Sector blocks
Factor = Factor.merge(perf,how='left',left_on='DT',right_on='DT')
Factor = Factor.merge(Sec_perf,how='left',left_on='DT',right_on='DT')
Factor["DT"]=pd.to_datetime(Factor["DT"]).dt.date
outputpath=r"E:\HKUST_CLASS\22spring\Investment_Model\project2\Rtn_fac.xlsx"
Factor.to_excel(outputpath,index="True",header="True")